In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
class ChessModel(nn.Module):
    def __init__(self):
        super(ChessModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 2 * 2, 512)  # Update this line based on the new shape
        self.fc2 = nn.Linear(512, 64)  # Output size depends on your move encoding

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        
        print(x.shape)  # Debugging print to check shape
        
        x = x.view(x.size(0), -1)  # Flatten the tensor dynamically
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
# Initialize model, loss function, and optimizer
model = ChessModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.eval()

ChessModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=256, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
)

In [10]:
# Dummy input to match the model's expected input size
# Assuming the input is a single-channel 8x8 chessboard
dummy_input = torch.randn(1, 1, 8, 8)

# Export the model to ONNX format
torch.onnx.export(
    model,                  # model being run
    dummy_input,            # model input (or a tuple for multiple inputs)
    "chess_model.onnx",     # where to save the model (can be a file or file-like object)
    export_params=True,     # store the trained parameter weights inside the model file
    opset_version=11,       # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names=['input'],     # the model's input names
    output_names=['output'],   # the model's output names
    dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                  'output': {0: 'batch_size'}}
)

print("Model has been exported to chess_model.onnx")

Model has been exported to chess_model.onnx
